## general_import.py transposed to a .ipynb file for debugging

In [6]:
prefix = 'IMPORT AS'

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
warnings.simplefilter(action='ignore', category=FutureWarning)

import io 
import shutil 
import re
import numpy as np
import pandas as pd
from copy import deepcopy
import os, sys
from pathlib import Path
import platform

from nanoHUB.application import Application
from nanoHUB.pipeline.SF_dataimports.file_handling import map_file
from nanoHUB.pipeline.SF_dataimports.data_handling import split_names, filter_bad_emails, strip_spaces_cols, rename_columns
from nanoHUB.pipeline.SF_dataimports.google_downloads import DefaultGoogleFactory

import time
from nanoHUB.logger import logger

log = logger('nanoHUB:google_imports')
pd.set_option('display.max_columns', 500)

application = Application.get_instance()
salesforce = application.new_salesforce_engine()
db_s = salesforce


from __future__ import print_function
import os.path
import os
import errno
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from apiclient import errors
from oauth2client.service_account import ServiceAccountCredentials
from httplib2 import Http

## stuff that's rather hard to find from documentation


Obtained Salesforce access token ...... True


## Setup GDrive API

In [7]:
downloader_factory = DefaultGoogleFactory(application, log)
downloader = downloader_factory.get_downloader()
dir_config = downloader_factory.get_folder_config()

to_import_file_paths, file_names, file_ids = downloader.process()

print(file_names)
print(file_ids)

[INFO] [google_downloads - nanoHUB:google_imports]: Found file: nH_CustSatSurvey2022_import.csv (1_DCw2-WSAR_s0-813VB3Sj-kF-ND35Yt) [google_downloads.process:99]
['nH_CustSatSurvey2022_import.csv']
['1_DCw2-WSAR_s0-813VB3Sj-kF-ND35Yt']


## Perform Sequential Imports

In [8]:
success_files = []
fail_files = []

sf_df = db_s.query_data('SELECT Id,nanoHUB_user_ID__c, Email FROM Contact') 
print(sf_df)

[Success] Bulk job creation successful. Job ID = 7505w00000cy4gcAAA
{"id":"7505w00000cy4gcAAA","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-27T01:39:48.000+0000","systemModstamp":"2022-01-27T01:39:49.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000cy4gcAAA","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-27T01:39:48.000+0000","systemModstamp":"2022-01-27T01:39:49.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":19435,"retries":0,"totalProcessingTime":1281}
{"id":"7505w00000cy4gcAAA","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-27T0

In [17]:
for file in to_import_file_paths:
    idf = pd.DataFrame()
    log.info("Reading %s" % file)
    try:
        idf = map_file(file, logger('nanoHUB:google_imports'))

        ## remove leading and trailing spaces ## add str.strip spaces for all columns and rename
        log.debug(idf.columns)
        idf = strip_spaces_cols(idf)

        idf, has_name = split_names(idf)
        if not has_name:
            log.info('No name in column `Name` needing to be split found')

        #rename columns
        idf = rename_columns(idf)
        if has_name == True:
            idf = idf.drop(columns='Name')#['NAME','LAST NAME','FIRST NAME'])            
        idf = idf.dropna(subset=['Email'])

        idf = filter_bad_emails(idf, log)

        # salesforce queries for contact data
        # deciding the queries
        import_df_cols = deepcopy(idf.columns)
        nh_id_flag = False
        email_flag = False
        if 'nanoHUB_user_ID__c' in import_df_cols:
            nh_id_flag = True

        if 'Email' in import_df_cols:
            email_flag = True    
        
        merged_df = idf.merge(sf_df, how = 'inner', on = ['Email'])
        
        prefixed_df = merged_df.filter(regex=re.compile(prefix, re.IGNORECASE))
        prefixed_df.columns = prefixed_df.columns.str.replace(prefix, "", case = False)
        prefixed_df.columns = prefixed_df.columns.str.lstrip(' ')
        log.info("Number of custom prefixed columns: %d" % len(prefixed_df.columns))
        log.debug(prefixed_df.columns)
        
        merged_df = merged_df[['Email','Id']]
        merged_df = merged_df.merge(prefixed_df, how = 'inner', left_index=True, right_index=True)
        
        log.info("Number of contacts: %d" % len(merged_df.index))
        log.debug(merged_df.head())
        log.debug(merged_df.tail())
        
        db_s_c = deepcopy(db_s)
        db_s_c.object_id = 'Contact'
        db_s_c.external_id = 'Id'
        db_s_c.send_data(merged_df)
        
        leads_df = idf[~idf['Email'].isin(sf_df['Email'])]
        log.info("Number of leads: %d" % len(leads_df))
        
    except Exception as e:
        log.error("Error with file: " + str(e))
        fail_files.append(file)
        

[INFO] [1138430544 - nanoHUB:google_imports]: Reading /home/saxenap/nanoHUB/.cache/Salesforce_Imports/To_Import/nH_CustSatSurvey2022_import.csv [1138430544.<module>:3]
[INFO] [file_handling - nanoHUB:google_imports]: File type .csv [file_handling.map_file:106]
[INFO] [api - charset_normalizer]: Detected a SIG or BOM mark on first 3 byte(s). Priority +1 given for utf_8. [api.from_bytes:159]
[INFO] [api - charset_normalizer]: Code page utf_8 is a multi byte encoding table and it appear that at least one character was encoded using n-bytes. [api.from_bytes:257]
[INFO] [api - charset_normalizer]: utf_8 passed initial chaos probing. Mean measured chaos is 0.000000 % [api.from_bytes:352]
[INFO] [api - charset_normalizer]: We detected language [('English', 1.0), ('Indonesian', 1.0), ('Simple English', 1.0)] using utf_8 [api.from_bytes:382]
[INFO] [api - charset_normalizer]: utf_8 is most likely the one. Stopping the process. [api.from_bytes:417]
[INFO] [file_handling - nanoHUB:google_imports]